In [1]:
! pip install datasets transformers[torch] 'accelerate>=0.26.0' peft bitsandbytes

  Using cached datasets-4.5.0-py3-none-any.whl.metadata (19 kB)
  Using cached accelerate-1.12.0-py3-none-any.whl.metadata (19 kB)
  Using cached peft-0.18.1-py3-none-any.whl.metadata (14 kB)
  Using cached bitsandbytes-0.49.2-py3-none-manylinux_2_24_x86_64.whl.metadata (10 kB)
  Using cached transformers-5.2.0-py3-none-any.whl.metadata (32 kB)
  Using cached pyarrow-23.0.1-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (3.1 kB)
  Using cached xxhash-3.6.0-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_28_x86_64.whl.metadata (13 kB)
  Using cached multiprocess-0.70.18-py312-none-any.whl.metadata (7.5 kB)
  Using cached huggingface_hub-1.4.1-py3-none-any.whl.metadata (13 kB)
  Using cached tokenizers-0.22.2-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.3 kB)
  Using cached safetensors-0.7.0-cp38-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.1 kB)
  Using cached hf_xet-1.2.0-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.

In [2]:
distilled_model = "../CustomProtocol/LA-Framework-dist-peft/results/distilled-Qwen3-1.7B/bs16-lr1e-05-G4-N1-NN1-lm1-len384-lora-8-32-0.1/pe2_rs0.5_nr128_ln_sr_tm0.2/1773"


In [9]:
from datasets import load_dataset

def load_io_dataset(split, uf_n=40):
    data_files = []
    data_files.append(f"TTE_uf{uf_n}/TTE_with_IO_{split}.json")
    dataset = load_dataset("json", data_files=f"TTE_uf{uf_n}/TTE_with_IO_{split}.json")

    def format_example(example):
        return {
            "text": f"Prompt: {example['input']}\nResponse: {example['output']}\n"
        }

    dataset = dataset["train"].map(format_example)
    return dataset

train_dataset = load_io_dataset("train")
test_dataset = load_io_dataset("test")
eval_dataset = load_io_dataset("eval")
print(train_dataset)
print(test_dataset)
print(eval_dataset)


Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/4430 [00:00<?, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/4430 [00:00<?, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/4430 [00:00<?, ? examples/s]

Dataset({
    features: ['input', 'output', 'text'],
    num_rows: 4430
})
Dataset({
    features: ['input', 'output', 'text'],
    num_rows: 4430
})
Dataset({
    features: ['input', 'output', 'text'],
    num_rows: 4430
})


In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

tokenizer = AutoTokenizer.from_pretrained(distilled_model)
model = AutoModelForCausalLM.from_pretrained(
    distilled_model,
    torch_dtype=torch.bfloat16,
    device_map="cuda"
)


`torch_dtype` is deprecated! Use `dtype` instead!


Loading weights:   0%|          | 0/310 [00:00<?, ?it/s]

In [12]:
prompt = eval_dataset[2]['input']
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
output = model.generate(**inputs, max_new_tokens=600, min_new_tokens=100)
print(tokenizer.decode(output[0], skip_special_tokens=True))

Based on the user's watching history and the film's rating, order the list of candidate films. Order the films where the first one of the list is the most likely to be watched by the user. The output should only contain the ordered list of the recommended films bounded by the  special strings '%% START RECOMMENDED LIST %%' and '%% END LIST %%'. Here follows the user's history and the list of candidate films.
%% START HISTORY %%
Movie name: Crocodile Dundee (1986)	Rating:4
Movie name: Indiana Jones and the Temple of Doom (1984)	Rating:4
Movie name: Ghost and the Darkness, The (1996)	Rating:4
Movie name: Three Musketeers, The (1993)	Rating:4
Movie name: Mission: Impossible (1996)	Rating:3
Movie name: Golden Child, The (1986)	Rating:4
Movie name: Monty Python and the Holy Grail (1974)	Rating:5
Movie name: Toy Story 2 (1999)	Rating:3
Movie name: Raising Arizona (1987)	Rating:4
Movie name: Being John Malkovich (1999)	Rating:3
%% END HISTORY %%
%% START CANDIDATES %%
Movie name: Sneakers (19

In [17]:
import torch
from tqdm.auto import tqdm   # optional, for nicer progress

inference_model = model

def split_list(text, start_token, end_token):
    if not (start_token + "\n" in text):
            return []
    phrase = text.split(start_token + "\n")
    movie_list = phrase[1].split("\n" + end_token)[0]
    movie_list = movie_list.split("\n")
    for i, movie in enumerate(movie_list):
        movie_list[i] = movie.split("\t")
    return movie_list


def perform_and_return_comparison(sample):
    #prompt = sample['text']
    inputs = tokenizer(sample['input'], return_tensors="pt").to("cuda")
    while True:
        output = inference_model.generate(**inputs, max_new_tokens=600)
        rating = tokenizer.decode(output[0], skip_special_tokens=True)
        if ("%% START RECOMMENDED LIST %%\n" in rating):
            break
    
    rating = split_list(rating, "%% START RECOMMENDED LIST %%", "%% END LIST %%")
    import gc
    gc.collect()
    torch.cuda.empty_cache()
    return {'input': sample['input'],
            'rating': rating}

In [ ]:
import torch
from tqdm.auto import tqdm   # optional, for nicer progress

# Parallel LLM generation of evaluation's results

def perform_and_return_comparison_batched(batch):
    # batch is a dict with lists: {'input': [...], 'text': [...]}
    inputs = tokenizer(
        batch['input'],
        return_tensors="pt",padding=True,
        truncation=True,
        max_length=1024,  # ← set sensible value, e.g. 2048
        padding_side='left'
    ).to("cuda")

    with torch.inference_mode():           # saves a bit of memory
        outputs = inference_model.generate(
            **inputs,
            max_new_tokens=600
        )

    decoded = tokenizer.batch_decode(
        outputs,
        skip_special_tokens=True
    )

    results = []
    for i, rating_full in enumerate(decoded):
        rating = split_list(
            rating_full,
            "%% START RECOMMENDED LIST %%",
            "%% END LIST %%"
        )
        results.append({
            'input': batch['input'][i],
            'rating': rating
        })

    return {'results': results}   # list of dicts — datasets will flatten it

# This is usually enough parallelism for one GPU
eval_output = eval_dataset.map(
    perform_and_return_comparison_batched,
    batched=True,
    batch_size=8,          # ← tune this: 4–16 most common sweet spot
                           # larger = better GPU util, but risk of OOM
    desc="Evaluating",
    remove_columns=eval_dataset.column_names,  # optional — keep only new fields
)

eval_output.to_json("peft_distilled_eval_processed_parallel.json")

Evaluating:   0%|          | 0/4430 [00:00<?, ? examples/s]